In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import random
import numpy as np

import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

df_1 = pd.read_csv("../output/task1-1.csv")
df_1_fill = pd.read_csv("../output/task1-1-fill.csv")
df_2 = pd.read_csv("../output/task1-2.csv")
df_2_fill = pd.read_csv("../output/task1-2-fill.csv")

In [3]:
plt.figure(figsize=(3, 3))
plt.plot(df_1['x'], df_1['y'], label='_task1-1', color='gray')
plt.plot(df_1['x'][0], df_1['y'][0], 'bx', label='Start', ms=10, mew=2)
plt.plot(df_1['x'][len(df_1)-1], df_1['y'][len(df_1)-1], 'rx', label='End', ms=10, mew=2)
plt.xlim(25,75)
plt.ylim(25,75)
plt.grid()
plt.margins(0)
plt.legend(loc='upper left')
plt.savefig("../../report/graphics/task1-1.pdf", bbox_inches='tight')
plt.close()

In [4]:
plt.figure(figsize=(3, 3))
plt.plot(df_2['x'], df_2['y'], label='_task1-2', color='gray')
plt.plot(df_2['x'][0], df_2['y'][0], 'bx', label='Start', ms=10, mew=2)
plt.plot(df_2['x'][len(df_2)-1], df_2['y'][len(df_2)-1], 'rx', label='End', ms=10, mew=2)
plt.xlim(25,75)
plt.ylim(25,75)
plt.grid()
plt.margins(0)
plt.legend(loc='upper left')
plt.savefig("../../report/graphics/task1-2.pdf", bbox_inches='tight')
plt.close()

In [5]:
plt.figure(figsize=(6, 6))
plt.plot(df_1_fill['x'], df_1_fill['y'], label='_task1-1-fill', color='gray', rasterized=True)
plt.plot(df_1_fill['x'][0], df_1_fill['y'][0], 'bx', label='Start', ms=10, mew=2)
plt.plot(df_1_fill['x'][len(df_1_fill)-1], df_1_fill['y'][len(df_1_fill)-1], 'rx', label='End', ms=10, mew=2)
plt.grid()
plt.margins(0)
plt.legend(loc='upper left')
plt.savefig("../../report/graphics/task1-1-fill.pdf", bbox_inches='tight')

In [6]:
plt.figure(figsize=(6, 6))
plt.plot(df_2_fill['x'], df_2_fill['y'], label='_task1-2-fill', color='gray', rasterized=True)
plt.plot(df_2_fill['x'][0], df_2_fill['y'][0], 'bx', label='Start', ms=10, mew=2)
plt.plot(df_2_fill['x'][len(df_2_fill)-1], df_2_fill['y'][len(df_2_fill)-1], 'rx', label='End', ms=10, mew=2)
plt.grid()
plt.margins(0)
plt.legend(loc='upper left')
plt.savefig("../../report/graphics/task1-2-fill.pdf", bbox_inches='tight')
plt.close()


In [7]:
# Load all .csv file
data = {}
for file in os.listdir("../output/bulk"):
    if file.endswith(".csv"):
        name = file.split(".")[0]
        record = {}
        record["df"] = pd.read_csv(f"../output/bulk/{file}")
        record["avg_length"] = record["df"]["length"].mean()
        record["avg_time"] = record["df"]["time"].mean()

        record["diagonal"] = name[6:7] == "2"
        record["start-x"] = int(name[8:10])
        record["start-y"] = int(name[11:13])
        record["end-x"] = int(name[15:17])
        record["end-y"] = int(name[18:20])
        record["distance"] = ((record["end-x"] - record["start-x"])**2 + (record["end-y"] - record["start-y"])**2)**0.5

        data[name] = record

In [8]:
# separate the data into diagonal and non-diagonal
diagonal_data = []
non_diagonal_data = []

for name, info in data.items():
    if info["diagonal"]:
        diagonal_data.append(info)
    else:
        non_diagonal_data.append(info)

# sort the data by distance
diagonal_data.sort(key=lambda x: x["distance"])
non_diagonal_data.sort(key=lambda x: x["distance"])

plt.figure(figsize=(8, 3))
plt.plot([x["distance"] for x in non_diagonal_data], [x["avg_length"] for x in non_diagonal_data], label="Square (steps)", color='blue')
plt.plot([x["distance"] for x in diagonal_data], [x["avg_length"] for x in diagonal_data], label="Diagonal (steps)", color='red')
plt.xlabel("Path length")
plt.ylabel("Average simulation steps")
plt.legend()

# plot time on opposite axis
plt.twinx()

plt.plot([x["distance"] for x in non_diagonal_data], [x["avg_time"] for x in non_diagonal_data], label="Square (time)", color='blue', linestyle='--')
plt.plot([x["distance"] for x in diagonal_data], [x["avg_time"] for x in diagonal_data], label="Diagonal (time)", color='red', linestyle='--')
plt.ylabel("Average simulation time (us)")
plt.ylim(0, 4500)

plt.margins(0)
plt.legend()
plt.savefig("../../report/graphics/task1-compare.pdf", bbox_inches='tight')

In [9]:
# calculate the multiplication factor between diagonal and non-diagonal
factor_length = []
for i in range(len(diagonal_data)):
    factor_length.append(diagonal_data[i]["avg_length"] / non_diagonal_data[i]["avg_length"])

factor_time = []
for i in range(len(diagonal_data)):
    factor_time.append(diagonal_data[i]["avg_time"] / non_diagonal_data[i]["avg_time"])

factor_length_avg = np.mean(factor_length)
factor_time_avg = np.mean(factor_time)

print(f"Average steps factor: {factor_length_avg}")
print(f"Average time factor: {factor_time_avg}")

plt.figure(figsize=(8, 3))
plt.plot([x["distance"] for x in diagonal_data], factor_length, label="_Steps factor", color='gray')
plt.plot([x["distance"] for x in diagonal_data], factor_time, label="_Time factor", color='gray')
plt.axhline(y=factor_length_avg, color='red', linestyle='--', label=f"Average steps  ")
plt.axhline(y=factor_time_avg, color='blue', linestyle='--', label=f"Average time factor")
plt.ylabel("Diagonal / Square")
plt.xlabel("Path length")
plt.ylim(0, 2.5)
plt.legend(loc='center right')
plt.savefig("../../report/graphics/task1-factor-length.pdf", bbox_inches='tight')

Average steps factor: 0.7393451245144123
Average time factor: 2.143568060901476


In [10]:
# select a random data
index = random.choice(range(len(diagonal_data)))
diagonal_selected = diagonal_data[index]
non_diagonal_selected = non_diagonal_data[index]

In [11]:
# calculate the average length of the path
avg_length_1a = non_diagonal_selected["df"]['length'].mean()
avg_length_2a = diagonal_selected["df"]['length'].mean()
print(avg_length_1a)
print(avg_length_2a)
print(f"Non Diagonal - ({non_diagonal_selected['start-x']}, {non_diagonal_selected['start-y']}) - ({non_diagonal_selected['end-x']}, {non_diagonal_selected['end-y']})")
print(f"Diagonal - ({diagonal_selected['start-x']}, {diagonal_selected['start-y']}) - ({diagonal_selected['end-x']}, {diagonal_selected['end-y']})")
plt.figure(figsize=(8, 3))
plt.plot(non_diagonal_selected["df"]['length'], color='dimgray', label='Square')
plt.plot(diagonal_selected["df"]['length'], color='lightgray', label='Diagonal')
plt.plot([0, non_diagonal_selected["df"]['length'].count()], [avg_length_1a, avg_length_1a], color='blue', label='Square (Avg)')
plt.plot([0, diagonal_selected["df"]['length'].count()], [avg_length_2a, avg_length_2a], color='red', label='Diagonal (Avg)')
plt.ylim(0, 500000)
plt.margins(0)
plt.legend(loc='upper left')
plt.xlabel("Simualtion iteration")
plt.ylabel("Simulation steps")
plt.savefig("../../report/graphics/task1-compare-single.pdf", bbox_inches='tight')

36331.1976
26328.5328
Non Diagonal - (52, 71) - (29, 36)
Diagonal - (52, 71) - (29, 36)


In [28]:
# calculate the mean at each step
rolling_length_1a = []
for i in range(1, non_diagonal_selected["df"]['length'].count()):
    rolling_length_1a.append(non_diagonal_selected["df"][:i]['length'].mean())
plt.figure(figsize=(8, 2))
plt.plot(rolling_length_1a, color='gray', label='_Square')
plt.plot([0, non_diagonal_selected["df"]['length'].count()], [avg_length_1a, avg_length_1a], label='Average simulation steps', color='red')
plt.text(non_diagonal_selected["df"]['length'].count() - 100, avg_length_1a - 5000, f'{avg_length_1a:.2f}', ha='right')
print(f"Non Diagonal - ({non_diagonal_selected['start-x']}, {non_diagonal_selected['start-y']}) - ({non_diagonal_selected['end-x']}, {non_diagonal_selected['end-y']})")
# plt.margins(0)
plt.legend(loc='upper right')
plt.xlabel("Simulation iteration")
plt.ylabel("Average simulation steps")
plt.ylim(0, 50000)
plt.savefig("../../report/graphics/task1-compare-square-roll.pdf", bbox_inches='tight')

Non Diagonal - (52, 71) - (29, 36)


In [26]:
# calculate the mean at each step
rolling_length_2a = []
for i in range(1, diagonal_selected["df"]['length'].count()):
    rolling_length_2a.append(diagonal_selected["df"][:i]['length'].mean())
plt.figure(figsize=(8, 2))
plt.plot(rolling_length_2a, color='gray', label='_Diagonal')
plt.plot([0, diagonal_selected["df"]['length'].count()], [avg_length_2a, avg_length_2a], label='Average simulation steps', color='red')
plt.text(diagonal_selected["df"]['length'].count() - 1000, avg_length_2a + 1000, "{:.3f}".format(avg_length_2a))
print(f"Diagonal - ({diagonal_selected['start-x']}, {diagonal_selected['start-y']}) - ({diagonal_selected['end-x']}, {diagonal_selected['end-y']})")
# plt.margins(0)
plt.legend(loc='upper right')
plt.xlabel("Simulation iteration")
plt.ylabel("Average simulation steps")
plt.ylim(0, 50000)
plt.savefig("../../report/graphics/task1-compare-diagonal-roll.pdf", bbox_inches='tight')

Diagonal - (52, 71) - (29, 36)


In [14]:
# open visited data
visited = pd.read_csv("../output/task1-visited.csv")
visited_np = visited.pivot(index='y', columns='x', values='count').to_numpy()
visited_diagonal = pd.read_csv("../output/task1-visited-diagonal.csv")
visited_diagonal_np = visited_diagonal.pivot(index='y', columns='x', values='count').to_numpy()

# get the minimum and maximum value for color normalization, which is the minimum and maximum value of the visited count
min_val = min(visited['count'].min(), visited_diagonal['count'].min())
max_val = max(visited['count'].max(), visited_diagonal['count'].max())

print(min_val, max_val)

878328 3413832


In [15]:
# plot the visited data
plt.figure(figsize=(4, 4))
plt.imshow(visited_np, cmap='hot')
plt.xlim(-10, 110)
plt.ylim(-10, 110)
plt.clim(min_val, max_val)
plt.colorbar(fraction=0.046, pad=0.04)
plt.savefig("../../report/graphics/task1-visited-square.pdf", bbox_inches='tight')
plt.close()

In [16]:
# plot the visited data
plt.figure(figsize=(4, 4))
plt.imshow(visited_diagonal_np, cmap='hot')
plt.xlim(-10, 110)
plt.ylim(-10, 110)
plt.clim(min_val, max_val)
plt.colorbar(fraction=0.046, pad=0.04)
plt.savefig("../../report/graphics/task1-visited-diagonal.pdf", bbox_inches='tight')
plt.close()

In [17]:
# do the difference between the two visited data
visited_diff_np = visited_diagonal_np - visited_np
plt.figure(figsize=(4, 4))
plt.imshow(visited_diff_np, cmap='hot', interpolation='nearest')
plt.xlim(-10, 110)
plt.ylim(-10, 110)
plt.colorbar(fraction=0.046, pad=0.04)
plt.savefig("../../report/graphics/task1-visited-diff.pdf", bbox_inches='tight')
plt.close()